# PARÁMETRO 13: TIPO DE NORMALIZACIÓN

Este notebook procesa **SOLO el Parámetro 13** de forma independiente.

**Configuración base (de Gonzalo):**
- IMG_SIZE = 256
- BATCH_SIZE = 4
- EPOCHS = 10
- FILTERS_BASE = 64

In [ ]:
%%javascript
function ClickConnect(){
  console.log("Manteniendo conexión activa...");
  document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect, 60000)  // Cada 60 segundos

In [ ]:
# ============================================================================
# IMPORTS Y SETUP INICIAL
# ============================================================================

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Conv2DTranspose, Activation, concatenate, Input, LayerNormalization
from tensorflow.keras import Model
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import json
import pickle
from datetime import datetime
import pandas as pd

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ============================================================================
# MÉTRICAS
# ============================================================================

def iou_metric(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return (intersection + smooth) / (union + smooth)

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (
        tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth
    )

def f1_score(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + smooth)
    recall = tp / (tp + fn + smooth)
    return 2 * precision * recall / (precision + recall + smooth)

In [ ]:
# ============================================================================
# EXPERIMENT MANAGER
# ============================================================================

class ExperimentManager:
    """Gestiona experimentos automáticamente"""

    def __init__(self, experiment_name, base_dir="/content/drive/MyDrive/Actividad2_Resultados_Damian"):
        self.experiment_name = experiment_name
        self.base_dir = base_dir
        self.experiment_dir = os.path.join(base_dir, experiment_name)
        os.makedirs(self.experiment_dir, exist_ok=True)

        self.models_dir = os.path.join(self.experiment_dir, "models")
        self.logs_dir = os.path.join(self.experiment_dir, "logs")
        self.plots_dir = os.path.join(self.experiment_dir, "plots")
        self.metrics_dir = os.path.join(self.experiment_dir, "metrics")

        for d in [self.models_dir, self.logs_dir, self.plots_dir, self.metrics_dir]:
            os.makedirs(d, exist_ok=True)

        print(f"✅ Experimento '{experiment_name}' inicializado")
        print(f"📁 Resultados en: {self.experiment_dir}")

    def run_experiment(self, model, train_dataset, test_dataset,
                       epochs, param_name, param_value):
        experiment_id = f"{param_name}_{param_value}"
        print(f"\n{'='*80}")
        print(f"🚀 EJECUTANDO: {param_name} = {param_value}")
        print(f"{'='*80}\n")

        model_path = os.path.join(self.models_dir, f"best_model_{experiment_id}.keras")
        checkpoint = ModelCheckpoint(
            model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1
        )
        early_stopping = EarlyStopping(
            monitor='val_loss', patience=20, mode='min', verbose=1, restore_best_weights=True
        )

        start_time = datetime.now()
        history = model.fit(
            train_dataset, epochs=epochs, validation_data=test_dataset,
            callbacks=[checkpoint, early_stopping], verbose=1
        )
        end_time = datetime.now()
        training_time = (end_time - start_time).total_seconds() / 60

        metrics = self._extract_metrics(history, param_name, param_value, training_time)
        self._save_all(history, metrics, experiment_id, param_name, param_value)

        print(f"\n✅ Completado en {training_time:.2f} min")
        return history, metrics

    def _extract_metrics(self, history, param_name, param_value, training_time):
        return {
            'experiment_name': self.experiment_name,
            'parameter_name': param_name,
            'parameter_value': param_value,
            'training_time_minutes': round(training_time, 2),
            'best_val_iou': float(max(history.history.get('val_iou_metric', [0]))),
            'best_val_dice': float(max(history.history.get('val_dice_coef', [0]))),
            'best_val_f1': float(max(history.history.get('val_f1_score', [0]))),
            'best_val_accuracy': float(max(history.history.get('val_accuracy', [0]))),
            'best_val_loss': float(min(history.history.get('val_loss', [999]))),
            'best_train_accuracy': float(max(history.history.get('accuracy', [0]))),
            'best_train_loss': float(min(history.history.get('loss', [999]))),
            'total_epochs': len(history.history['loss'])
        }

    def _save_all(self, history, metrics, experiment_id, param_name, param_value):
        with open(os.path.join(self.metrics_dir, f"metrics_{experiment_id}.json"), 'w') as f:
            json.dump(metrics, f, indent=4)

        with open(os.path.join(self.logs_dir, f"history_{experiment_id}.pkl"), 'wb') as f:
            pickle.dump(history.history, f)

        with open(os.path.join(self.logs_dir, f"epoch_logs_{experiment_id}.txt"), 'w') as f:
            f.write(f"LOGS DE ENTRENAMIENTO - {param_name} = {param_value}\n")
            f.write("="*80 + "\n\n")
            for epoch in range(len(history.history['loss'])):
                f.write(f"Epoch {epoch+1}\n")
                f.write(f"  Loss: {history.history['loss'][epoch]:.6f}\n")
                f.write(f"  Val Loss: {history.history['val_loss'][epoch]:.6f}\n")
                f.write(f"  Accuracy: {history.history['accuracy'][epoch]:.6f}\n")
                f.write(f"  Val Accuracy: {history.history['val_accuracy'][epoch]:.6f}\n\n")

    def compare_experiments(self, histories_dict, param_name):
        metrics = ['loss', 'accuracy', 'iou_metric', 'dice_coef', 'f1_score']
        fig, axes = plt.subplots(len(metrics), 2, figsize=(15, 5*len(metrics)))

        for i, metric in enumerate(metrics):
            for param_value, history in histories_dict.items():
                if metric in history.history:
                    axes[i, 0].plot(history.history[metric],
                                   label=f'{param_name}={param_value}', marker='o', markersize=4)
            axes[i, 0].set_title(f'Training {metric.upper()}', fontweight='bold')
            axes[i, 0].set_xlabel('Epoch')
            axes[i, 0].set_ylabel(metric.replace('_', ' ').title())
            axes[i, 0].legend()
            axes[i, 0].grid(True, alpha=0.3)

            val_metric = f'val_{metric}'
            for param_value, history in histories_dict.items():
                if val_metric in history.history:
                    axes[i, 1].plot(history.history[val_metric],
                                   label=f'{param_name}={param_value}', marker='o', markersize=4)
            axes[i, 1].set_title(f'Validation {metric.upper()}', fontweight='bold')
            axes[i, 1].set_xlabel('Epoch')
            axes[i, 1].set_ylabel(metric.replace('_', ' ').title())
            axes[i, 1].legend()
            axes[i, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plot_path = os.path.join(self.plots_dir, f"comparison_{param_name}.png")
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"\n📈 Gráfica guardada en: {plot_path}")
        plt.show()

    def generate_summary_table(self, metrics_list):
        df = pd.DataFrame(metrics_list)
        df = df[['parameter_name', 'parameter_value', 'best_val_iou', 'best_val_dice',
                 'best_val_f1', 'best_val_accuracy', 'best_val_loss', 'training_time_minutes']]
        df.columns = ['Parámetro', 'Valor', 'Val IoU', 'Val Dice', 'Val F1',
                      'Val Accuracy', 'Val Loss', 'Tiempo (min)']

        df.to_csv(os.path.join(self.metrics_dir, "summary_table.csv"), index=False)
        with open(os.path.join(self.metrics_dir, "summary_table.txt"), 'w') as f:
            f.write(df.to_string(index=False))

        print("\n📋 TABLA RESUMEN:")
        print(df.to_string(index=False))
        return df

In [ ]:
# ============================================================================
# FUNCIONES DE CARGA
# ============================================================================

def load_data(path, img_size):
    images = []
    masks = []
    num_images = 100

    for img_files, mask_files in zip(
        os.listdir(path + "/Original")[:num_images],
        os.listdir(path + "/Ground truth")[:num_images]
    ):
        img_path = os.path.join(path + "/Original", img_files)
        mask_path = os.path.join(path + "/Ground truth", mask_files)

        img = load_img(img_path, target_size=img_size)
        mask = load_img(mask_path, target_size=img_size, color_mode="grayscale")

        img = img_to_array(img) / 255.0
        mask = img_to_array(mask) / 255.0

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

def tf_dataset(images, masks, batch_size=4, augment_fn=None):
    def generator():
        for img, mask in zip(images, masks):
            if augment_fn:
                img, mask = augment_fn(img, mask)
            yield img, mask

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(256, 256, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(256, 256, 1), dtype=tf.float32)
        )
    )
    return dataset.batch(batch_size)

In [ ]:
# ============================================================================
# ARQUITECTURAS UNET CON DIFERENTES NORMALIZACIONES
# ============================================================================

def conv_block_batch(input, filters, kernel_size=3):
    """Con BatchNormalization"""
    x = Conv2D(filters, kernel_size, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters, kernel_size, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def conv_block_none(input, filters, kernel_size=3):
    """Sin normalización"""
    x = Conv2D(filters, kernel_size, padding="same")(input)
    x = Activation("relu")(x)
    x = Conv2D(filters, kernel_size, padding="same")(x)
    x = Activation("relu")(x)
    return x

def conv_block_layer(input, filters, kernel_size=3):
    """Con LayerNormalization"""
    x = Conv2D(filters, kernel_size, padding="same")(input)
    x = LayerNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters, kernel_size, padding="same")(x)
    x = LayerNormalization()(x)
    x = Activation("relu")(x)
    return x

def encoder_block_custom(input, filters, conv_block_fn, kernel_size=3):
    x = conv_block_fn(input, filters, kernel_size)
    p = MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block_custom(input, skip, filters, conv_block_fn, kernel_size=3):
    x = Conv2DTranspose(filters, 2, strides=2, padding="same")(input)
    x = concatenate([x, skip])
    x = conv_block_fn(x, filters, kernel_size)
    return x

def build_model_4_layers(img_size=256, filters_base=64, conv_block_fn=conv_block_batch, kernel_size=3):
    input_layer = Input(shape=(img_size, img_size, 3))

    x1, p1 = encoder_block_custom(input_layer, filters_base, conv_block_fn, kernel_size)
    x2, p2 = encoder_block_custom(p1, filters_base * 2, conv_block_fn, kernel_size)
    x3, p3 = encoder_block_custom(p2, filters_base * 4, conv_block_fn, kernel_size)
    x4, p4 = encoder_block_custom(p3, filters_base * 8, conv_block_fn, kernel_size)

    a1 = conv_block_fn(p4, filters_base * 16, kernel_size)

    d1 = decoder_block_custom(a1, x4, filters_base * 8, conv_block_fn, kernel_size)
    d2 = decoder_block_custom(d1, x3, filters_base * 4, conv_block_fn, kernel_size)
    d3 = decoder_block_custom(d2, x2, filters_base * 2, conv_block_fn, kernel_size)
    d4 = decoder_block_custom(d3, x1, filters_base, conv_block_fn, kernel_size)

    output = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d4)
    return Model(input_layer, output)

In [ ]:
# ============================================================================
# CARGAR DATOS (UNA SOLA VEZ)
# ============================================================================

import kagglehub
download_path = kagglehub.dataset_download("nikitamanaenkov/fundus-image-dataset-for-vessel-segmentation")
print("Path to dataset files:", download_path)

# Cargar datos
IMG_SIZE = 256
print(f"📥 Cargando datos con IMG_SIZE={IMG_SIZE}...")
x_train, y_train = load_data(f"{download_path}/train", img_size=(IMG_SIZE, IMG_SIZE))
x_test, y_test = load_data(f"{download_path}/test", img_size=(IMG_SIZE, IMG_SIZE))
print(f"✅ Datos cargados: train {x_train.shape}, test {x_test.shape}")

In [ ]:
# ============================================================================
# DEFINIR CONSTANTES
# ============================================================================

BATCH_SIZE = 4   # Mejor de Gonzalo
EPOCHS = 10      # Mejor de Gonzalo
FILTERS_BASE = 64

print(f"\n📊 Configuración:")
print(f"  - IMG_SIZE: {IMG_SIZE}")
print(f"  - BATCH_SIZE: {BATCH_SIZE}")
print(f"  - EPOCHS: {EPOCHS}")
print(f"  - FILTERS_BASE: {FILTERS_BASE}")

In [ ]:
# ============================================================================
# PARÁMETRO 13: TIPO DE NORMALIZACIÓN
# ============================================================================

print("\n" + "="*80)
print("PARÁMETRO 13: TIPO DE NORMALIZACIÓN")
print("="*80)

manager13 = ExperimentManager("Parametro_13_Normalization")

normalization_configs = {
    'Batch': conv_block_batch,
    'None': conv_block_none,
    'Layer': conv_block_layer
}

histories_norm = {}
metrics_list_norm = []

train_dataset = tf_dataset(x_train, y_train, batch_size=BATCH_SIZE)
test_dataset = tf_dataset(x_test, y_test, batch_size=BATCH_SIZE)

for norm_name, conv_block_fn in normalization_configs.items():
    print(f"\n🔄 Probando normalización: {norm_name}")

    model = build_model_4_layers(img_size=IMG_SIZE, filters_base=FILTERS_BASE, conv_block_fn=conv_block_fn)
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy", iou_metric, dice_coef, f1_score]
    )

    history, metrics = manager13.run_experiment(
        model=model,
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        epochs=EPOCHS,
        param_name="normalization",
        param_value=norm_name
    )

    histories_norm[norm_name] = history
    metrics_list_norm.append(metrics)

manager13.compare_experiments(histories_norm, param_name="normalization")
summary_df_norm = manager13.generate_summary_table(metrics_list_norm)

In [ ]:
# ============================================================================
# VISUALIZACIÓN CUALITATIVA - PARÁMETRO 13: normalization
# ============================================================================

import matplotlib.pyplot as plt
import numpy as np

print("\n🖼️ Generando visualización cualitativa...")

# Seleccionar una imagen de test
sample_index = 0
test_image = x_test[sample_index]
test_mask = y_test[sample_index]

# Crear figura
num_models = len(histories_norm)
fig, axes = plt.subplots(1, num_models + 2, figsize=(4 * (num_models + 2), 4))

# Imagen original
axes[0].imshow(test_image)
axes[0].set_title('Original', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Ground truth
axes[1].imshow(test_mask[:, :, 0], cmap='gray')
axes[1].set_title('Ground Truth', fontsize=12, fontweight='bold')
axes[1].axis('off')

# Predicciones de cada modelo
for idx, (param_value, history) in enumerate(histories_norm.items()):
    # Cargar el mejor modelo guardado
    model_path = os.path.join(manager13.models_dir, f"best_model_normalization_{param_value}.keras")

    try:
        model = tf.keras.models.load_model(
            model_path,
            custom_objects={
                'iou_metric': iou_metric,
                'dice_coef': dice_coef,
                'f1_score': f1_score
            }
        )

        # Predecir
        test_image_expanded = np.expand_dims(test_image, axis=0)
        prediction = model.predict(test_image_expanded, verbose=0)[0]

        # Mostrar
        axes[idx + 2].imshow(prediction[:, :, 0], cmap='gray', vmin=0, vmax=1)
        axes[idx + 2].set_title(f'{param_value}', fontsize=12, fontweight='bold')
        axes[idx + 2].axis('off')

    except Exception as e:
        print(f"⚠️ Error cargando modelo {param_value}: {e}")
        axes[idx + 2].text(0.5, 0.5, 'Error', ha='center', va='center')
        axes[idx + 2].set_title(f'{param_value}', fontsize=12, fontweight='bold')
        axes[idx + 2].axis('off')

plt.suptitle(f'Comparación Cualitativa - Normalization (Batch, None, Layer)',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()

# Guardar
visual_path = os.path.join(manager13.plots_dir, "visual_comparison_normalization.png")
plt.savefig(visual_path, dpi=300, bbox_inches='tight')
print(f"✅ Visualización guardada en: {visual_path}")
plt.show()

print("\n" + "="*80)
print("✅ PARÁMETRO 13 COMPLETADO")
print("="*80)